# Keras Sandbox

## Notes and Sources
- Contrastive Loss
- Triplet Loss
https://github.com/maciejkula/triplet_recommendations_keras
- Comparison
https://www.quora.com/What-are-the-advantages-of-using-a-triplet-loss-function-over-a-contrastive-loss-How-would-you-decide-which-to-use

In [1]:
import keras
import pandas as pd

/Users/ischeinfeld/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def load_data(fname, test_split = 0.2):
    print 'Loading data...'
    df = pd.read_csv(config.Data.feature_matrix)
    df['encoded_msg'] = df['encoded_msg'].apply(lambda x: \
        [int(e) for e in x.split()])
    df = df.reindex(random.permutation(df.index))

    X_train = df['encoded_msg'].values[:int(len(df) * (1 - test_split))]
    y_train = array(df['was_blocked'].values[:int(len(df) * (1 - test_split))])

    X_test = array(df['encoded_msg'].values[int(len(df) * (1 - test_split))])
    y_test = array(df['was_blocked'].values[int(len(df) * (1 - test_split))])

    #return pad_sequences(X_train) ,y_train, X_test, y_test
    return X_train ,y_train, X_test, y_test